# Sidekick with Planner and Clarification

This enhanced version includes:
- 🤔 Clarifying questions before starting work
- 📋 Planner agent that creates execution plans
- 🛠️ Full toolkit (browser, files, search, notifications, Python, Wikipedia)

In [ ]:
from typing import Annotated, TypedDict, List, Dict, Any, Optional
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from IPython.display import Image, display
import gradio as gr
from dotenv import load_dotenv
from datetime import datetime
from playwright.async_api import async_playwright
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
import os
import requests
from langchain.agents import Tool
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_experimental.tools import PythonREPLTool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

In [ ]:
load_dotenv(override=True)

### Tools

In [ ]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"
serper = GoogleSerperAPIWrapper()

async def playwright_tools():
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless=False)
    toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=browser)
    return toolkit.get_tools(), browser, playwright

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data={"token": pushover_token, "user": pushover_user, "message": text})
    return "success"

def get_file_tools():
    toolkit = FileManagementToolkit(root_dir="sandbox")
    return toolkit.get_tools()

async def other_tools():
    push_tool = Tool(name="send_push_notification", func=push, description="Use this tool when you want to send a push notification")
    file_tools = get_file_tools()
    tool_search = Tool(
        name="search",
        func=serper.run,
        description="Use this tool when you want to get the results of an online web search"
    )
    wikipedia = WikipediaAPIWrapper()
    wiki_tool = WikipediaQueryRun(api_wrapper=wikipedia)
    python_repl = PythonREPLTool()
    
    return file_tools + [push_tool, tool_search, python_repl, wiki_tool]

### Structured Outputs

In [ ]:
class ClarifyingQuestion(BaseModel):
    question: str = Field(description="The clarifying question to ask")
    why_important: str = Field(description="Why this question matters")

class ClarifyingQuestions(BaseModel):
    questions: List[ClarifyingQuestion] = Field(description="List of 2-4 clarifying questions")

class PlanStep(BaseModel):
    step_number: int = Field(description="The step number")
    action: str = Field(description="What needs to be done in this step")
    tools_needed: List[str] = Field(description="List of tools that might be needed")

class ExecutionPlan(BaseModel):
    steps: List[PlanStep] = Field(description="Ordered list of steps to complete the task")
    estimated_complexity: str = Field(description="Low, Medium, or High complexity")

class EvaluatorOutput(BaseModel):
    feedback: str = Field(description="Feedback on the assistant's response")
    success_criteria_met: bool = Field(description="Whether the success criteria have been met")
    user_input_needed: bool = Field(description="True if more input is needed from the user")

### State

In [ ]:
class State(TypedDict):
    messages: Annotated[List[Any], add_messages]
    success_criteria: str
    clarifications: Optional[str]
    execution_plan: Optional[str]
    feedback_on_work: Optional[str]
    success_criteria_met: bool
    user_input_needed: bool
    stage: str  # "clarifying", "planning", "executing"

### Setup and Configuration

In [ ]:
# Store everything in one dict
state = {}

async def setup():
    """Initialize tools and LLMs"""
    # Setup tools
    playwright_tools_list, state['browser'], state['playwright'] = await playwright_tools()
    other_tools_list = await other_tools()
    state['tools'] = playwright_tools_list + other_tools_list
    
    # Setup LLMs with structured outputs
    worker_llm = ChatOpenAI(model="gpt-4o-mini")
    state['worker_llm_with_tools'] = worker_llm.bind_tools(state['tools'])
    
    clarifying_llm = ChatOpenAI(model="gpt-4o-mini")
    state['clarifying_llm'] = clarifying_llm.with_structured_output(ClarifyingQuestions)
    
    planner_llm = ChatOpenAI(model="gpt-4o-mini")
    state['planner_llm'] = planner_llm.with_structured_output(ExecutionPlan)
    
    evaluator_llm = ChatOpenAI(model="gpt-4o-mini")
    state['evaluator_llm'] = evaluator_llm.with_structured_output(EvaluatorOutput)
    
    state['memory'] = MemorySaver()
    
    await build_graph()

def cleanup():
    """Cleanup browser resources"""
    if 'browser' in state and state['browser']:
        import asyncio
        try:
            loop = asyncio.get_running_loop()
            loop.create_task(state['browser'].close())
            if 'playwright' in state and state['playwright']:
                loop.create_task(state['playwright'].stop())
        except:
            pass

### Core Nodes

In [ ]:
def clarifying_agent(graph_state: State) -> Dict[str, Any]:
    """Ask clarifying questions about the task"""
    user_request = graph_state["messages"][0].content
    
    prompt = f"""User's request: {user_request}

    Success criteria: {graph_state['success_criteria']}

    Generate 2-4 clarifying questions to better understand the user's needs.
    Focus on scope, preferences, constraints, and desired output format."""
    
    questions = state['clarifying_llm'].invoke([HumanMessage(content=prompt)])
    
    # Format questions
    response = "Before I start, I have some clarifying questions:\n\n"
    for i, q in enumerate(questions.questions, 1):
        response += f"**{i}. {q.question}**\n"
    
    return {
        "messages": [AIMessage(content=response)],
        "stage": "planning"
    }

def planner_agent(graph_state: State) -> Dict[str, Any]:
    """Create an execution plan for the task"""
    user_request = graph_state["messages"][0].content
    clarifications = graph_state.get("clarifications", "No additional clarifications")
    
    available_tools = ", ".join([tool.name for tool in state['tools']])
    
    prompt = f"""User's request: {user_request}

    Clarifications: {clarifications}

    Success criteria: {graph_state['success_criteria']}

    Available tools: {available_tools}

    Create a detailed execution plan with ordered steps to accomplish this task.
    Consider which tools are needed for each step."""
        
    plan = state['planner_llm'].invoke([HumanMessage(content=prompt)])
    
    # Format plan
    plan_text = f"Execution Plan (Complexity: {plan.estimated_complexity}):\n\n"
    for step in plan.steps:
        plan_text += f"Step {step.step_number}: {step.action}\n"
        if step.tools_needed:
            plan_text += f"  Tools: {', '.join(step.tools_needed)}\n"
    
    return {
        "messages": [AIMessage(content=plan_text)],
        "execution_plan": plan_text,
        "stage": "executing"
    }

def worker(graph_state: State) -> Dict[str, Any]:
    """Execute the plan using tools"""
    system_message = f"""You are a helpful assistant executing a planned task.

    Current date and time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

    Success criteria: {graph_state['success_criteria']}

    Execution plan:
    {graph_state.get('execution_plan', 'No formal plan')}

    Work through the plan systematically. Use tools as needed.
    When finished, provide a clear final response."""
    
    if graph_state.get("feedback_on_work"):
        system_message += f"\n\nPrevious feedback: {graph_state['feedback_on_work']}\nPlease address this feedback."
    
    messages = [SystemMessage(content=system_message)] + graph_state["messages"][1:]
    response = state['worker_llm_with_tools'].invoke(messages)
    
    return {"messages": [response]}

def evaluator(graph_state: State) -> Dict[str, Any]:
    """Evaluate if task is complete"""
    last_response = graph_state["messages"][-1].content
    
    prompt = f"""Evaluate this assistant's work:

    Success criteria: {graph_state['success_criteria']}

    Assistant's response: {last_response}

    Decide if:
    1. Success criteria is met
    2. More user input is needed
    3. Provide constructive feedback"""
    
    eval_result = state['evaluator_llm'].invoke([HumanMessage(content=prompt)])
    
    return {
        "messages": [AIMessage(content=f"Evaluation: {eval_result.feedback}")],
        "feedback_on_work": eval_result.feedback,
        "success_criteria_met": eval_result.success_criteria_met,
        "user_input_needed": eval_result.user_input_needed
    }

## Graph

In [ ]:
def worker_router(graph_state: State) -> str:
    """Route from worker to tools or evaluator"""
    last_message = graph_state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return "evaluator"

def route_after_evaluation(graph_state: State) -> str:
    """Route after evaluation - continue or end"""
    if graph_state["success_criteria_met"] or graph_state["user_input_needed"]:
        return "END"
    return "worker"

def route_from_start(graph_state: State) -> str:
    """Route based on current stage"""
    stage = graph_state.get("stage", "clarifying")
    if stage == "clarifying":
        return "clarifying"
    elif stage == "planning":
        return "planning"
    return "worker"

async def build_graph():
    """Build the LangGraph workflow"""
    graph_builder = StateGraph(State)
    
    # Add nodes
    graph_builder.add_node("clarifying", clarifying_agent)
    graph_builder.add_node("planning", planner_agent)
    graph_builder.add_node("worker", worker)
    graph_builder.add_node("tools", ToolNode(tools=state['tools']))
    graph_builder.add_node("evaluator", evaluator)
    
    # Add edges
    graph_builder.add_conditional_edges(
        START, 
        route_from_start, 
        {"clarifying": "clarifying", "planning": "planning", "worker": "worker"}
    )
    graph_builder.add_edge("clarifying", END)
    graph_builder.add_edge("planning", "worker")
    graph_builder.add_conditional_edges(
        "worker", 
        worker_router, 
        {"tools": "tools", "evaluator": "evaluator"}
    )
    graph_builder.add_edge("tools", "worker")
    graph_builder.add_conditional_edges(
        "evaluator", 
        route_after_evaluation,
        {"worker": "worker", "END": END}
    )
    
    state['graph'] = graph_builder.compile(checkpointer=state['memory'])

In [ ]:
from IPython.display import Image, display

# First run setup to build the graph
await setup()

# Then display the graph visualization
display(Image(state['graph'].get_graph().draw_mermaid_png()))

### Chat Handler

In [ ]:
async def chat(message: str, history: List, success_criteria: str):
    """Simplified chat handler"""
    
    if 'graph' not in state:
        return history + [{"role": "assistant", "content": "System initializing... Please run await setup() first."}]
    
    # Use a default thread ID
    config = {"configurable": {"thread_id": "default"}}
    
    # Add user message
    history = history + [{"role": "user", "content": message}]
    
    # Stage 1: Initial request -> Ask clarifying questions
    if len(history) == 1:
        graph_state = {
            "messages": [HumanMessage(content=message)],
            "success_criteria": success_criteria or "Task completed accurately and thoroughly",
            "stage": "clarifying"
        }
        result = await state['graph'].ainvoke(graph_state, config=config)
        return history + [{"role": "assistant", "content": result["messages"][-1].content}]
    
    # Stage 2: Answer clarifying questions -> Create plan
    elif "clarifying questions" in history[-2]["content"].lower():
        original_request = history[0]["content"]
        graph_state = {
            "messages": [HumanMessage(content=original_request)],
            "success_criteria": success_criteria or "Task completed accurately and thoroughly",
            "clarifications": message,
            "stage": "planning"
        }
        result = await state['graph'].ainvoke(graph_state, config=config)
        return history + [{"role": "assistant", "content": result["messages"][-1].content + "\n\nType 'proceed' to start execution."}]
    
    # Stage 3: Review plan -> Execute
    elif "execution plan" in history[-2]["content"].lower():
        if "proceed" in message.lower():
            original_request = history[0]["content"]
            clarifications = history[2]["content"]
            plan = history[-2]["content"]
            
            graph_state = {
                "messages": [HumanMessage(content=original_request)],
                "success_criteria": success_criteria or "Task completed accurately and thoroughly",
                "clarifications": clarifications,
                "execution_plan": plan,
                "stage": "executing",
                "success_criteria_met": False,
                "user_input_needed": False
            }
            
            result = await state['graph'].ainvoke(graph_state, config=config)
            final_content = f"{result['messages'][-2].content}\n\n---\n{result['messages'][-1].content}"
            return history + [{"role": "assistant", "content": final_content}]
        else:
            return history + [{"role": "assistant", "content": "Type 'proceed' when ready to execute."}]
    
    # Stage 4: New request
    else:
        graph_state = {
            "messages": [HumanMessage(content=message)],
            "success_criteria": success_criteria or "Task completed accurately and thoroughly",
            "stage": "clarifying"
        }
        result = await state['graph'].ainvoke(graph_state, config=config)
        return history + [{"role": "assistant", "content": result["messages"][-1].content}]

### UI

In [ ]:
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue")) as demo:
    gr.Markdown("# 🤖 Enhanced Sidekick")
    gr.Markdown("AI assistant with clarifying questions and execution planning")
    
    chatbot = gr.Chatbot(label="Sidekick", height=500, type="messages")
    
    with gr.Row():
        msg_input = gr.Textbox(
            label="Message",
            placeholder="What would you like me to help with?",
            lines=2,
            scale=4
        )
        send_btn = gr.Button("Send", variant="primary", scale=1)
    
    success_input = gr.Textbox(
        label="Success Criteria (optional)",
        placeholder="What defines success for this task?",
        lines=1
    )
    
    clear_btn = gr.Button("Clear Chat", variant="secondary")
    
    # Wire up events
    send_btn.click(
        chat,
        inputs=[msg_input, chatbot, success_input],
        outputs=[chatbot]
    )
    
    msg_input.submit(
        chat,
        inputs=[msg_input, chatbot, success_input],
        outputs=[chatbot]
    )
    
    clear_btn.click(lambda: [], outputs=[chatbot])

# Launch
demo.launch(inbrowser=True)